# Custom Evaluator (Manhattan Distance)

In this example we will see how we can create a custom evaluator. This evaluator will calculate the Manhattan distance.

#### Import Statements

In [ ]:
import numpy as np

from pytorch_wrapper.evaluators import AbstractEvaluator, GenericEvaluatorResults


#### Evaluator definition

In order to create a custom evaluator we need to inherit from `pytorch_wrapper.evaluators.AbstractEvaluator` and implement the `reset`, `step`, and `calculate` methods. The `reset`  method is called at the begining of each epoch and in the begining of the `System`'s evaluate method and must reset the evaluator. The `step` method is called for each batch and must store the information needed for the calculation of the metric. Finally the `calculate` method must calculate the metric and return a `pytorch_wrapper.evaluators.GenericEvaluatorResults` object or any other subclass of `pytorch_wrapper.evaluators.AbstractEvaluatorResults`.

In [ ]:
class ManhattanDistanceEvaluator(AbstractEvaluator):
    def __init__(self, model_output_key=None, batch_target_key='target'):
        super(ManhattanDistanceEvaluator, self).__init__()
        self._model_output_key = model_output_key
        self._batch_target_key = batch_target_key
        self.reset()

    def reset(self):
        self._outputs = []
        self._targets = []

    def step(self, output, batch, last_activation=None):
        if self._model_output_key is not None:
            output = output[self._model_output_key]

        self._outputs.extend(output.tolist())
        self._targets.extend(batch[self._batch_target_key].tolist())

    def calculate(self):
        mean_distance = np.sum(np.abs(np.array(self._outputs) - np.array(self._targets)))
        return GenericEvaluatorResults(score=mean_distance,
                                       label='manh-dist',
                                       score_format='%f',
                                       is_max_better=False)
